
# Notebook 1: Meshes

<div style="float: right; width: 40%">
    
![](media/SampleSphericalMesh.png)

</div>



This notebookIntroduces the mesh discretisation that we use in `Underworld3` and how you can build one of the pre-defined meshes. This notebook also show you how to use the `pyvista` visualisation tools for `Underworld3` objects. The mesh holds information on the mesh geometry, boundaries and coordinate systems and you can attach data to the mesh (see Notebook 2: [Variables](2-Variables.ipynb)). 


In [ ]:
#|  echo: false

# This is required to fix pyvista 
# (visualisation) crashes in interactive notebooks (including on binder)

import nest_asyncio
nest_asyncio.apply()

In [ ]:
#| output: false # Suppress warnings in html version

import underworld3 as uw
import numpy as np
import sympy

#### Underworld meshing module

Underworld can read mesh definition files from the `gmsh` package but there are
some constraints on how to specify boundaries if those meshes are to be used
to solve numerical problems. 

The `underworld.meshing` module has a collection of `gmsh` (python) examples for
common, simple meshes. 

In [ ]:
mesh = uw.meshing.CubedSphere(
    radiusOuter=1.0,
    radiusInner=0.547,
    numElements=8,
    refinement=0,
    simplex=True,
    verbose=True,
)

#### Mesh coordinate arrays

If you need to check the physical coordinates of the mesh, there
is a data array 

```python
    mesh.X.coords
```

which is a read-only `numpy` view of the coordinates (on the local
segment of the mesh when running in parallel)


In [ ]:
mesh.X.coords

There are other pre-built meshes you can try. This is a cuboid divided into regular tetrahedra:

```python
mesh_usb = uw.meshing.UnstructuredSimplexBox(
    minCoords = (-1.0, -1.0, -1.0),
    maxCoords = (+1.0, +1.0, +1.0),
    cellSize = 0.2,
    regular=True,
    verbose=False,
)
```

and this is a two-dimensional annulus mesh


```python
mesh_ann = uw.meshing.Annulus(
    radiusOuter=1.0,
    radiusInner=0.547,
    cellSize= 0.5,
    cellSizeOuter=0.033,
    cellSizeInner=0.05,
    verbose=False,
)
```

The meshing infrastructure for underworld3 is documented here: https://underworldcode.github.io/underworld3/main_api/underworld3/meshing.html

In [ ]:
import pyvista as pv
import underworld3.visualisation as vis

# Try out each one !

pvmesh = vis.mesh_to_pv_mesh(mesh)
pvmesh.point_data["z"] = vis.scalar_fn_to_pv_points(pvmesh, mesh.CoordinateSystem.X[2])
pvmesh1 = pvmesh.copy()

if mesh.dim==3:
    pvmesh_c = pvmesh.clip( normal='z', crinkle=True, inplace=False, origin=(0.0,0.0,0.01))

pl = pv.Plotter(window_size=(750, 750))
pl.add_mesh(pvmesh_c, show_edges=True, show_scalar_bar=False, opacity=1.0)
pl.add_mesh(pvmesh1, show_edges=True, show_scalar_bar=False, opacity=0.3)


# Save and show the mesh
pl.export_html("html5/spherical_mesh_plot.html") 

In [ ]:
#| fig-cap: "Interactive Image: Spherical shell mesh cut in half and overlain with transparent view of the whole mesh. Cubed sphere discretisation using hexahedral elements"

from IPython.display import IFrame
IFrame(src="html5/spherical_mesh_plot.html", width=600, height=400)

### Coordinate systems

The mesh has an associated "natural" coordinate system (usually Cartesian), but it may also have other, more convenient, coordinate systems. 

For example, the spherical mesh above has a Cartesian coordinate system which is the one used to navigate the mesh and describe the location of each point. It also has a spherical $(r, \theta, \phi)$ system which is **symbolic** and can be expanded in terms of the Cartesian coordinates.

In [ ]:
## The coordinate system

X = mesh.CoordinateSystem.X
R = mesh.CoordinateSystem.R

display(X)
display(R)
display(uw.unwrap(uw.function.expression, R))

### Mesh information

`mesh.view()` allows you to interrogate the mesh to identify the 
mesh data structures (which means you can find by name any variable that
is automatically constructed by, for example, one of the numerical solvers).

It also identifies boundaries of the mesh and their sizes when distributed
in parallel. There is a `PETSc` equivalent which is also called and this 
contains low-level information on the mesh topology.

In [ ]:
mesh.view(1)

### Mesh deformation

You can adjust the coordinates using:

```python
    mesh.deform(local_coordinate_array)
```
This rebuilds all the finite element gadgets that live on the mesh but it
will not do any remeshing of the points. It is useful for small deformation
such as following a free surface but not large-deformation adaptive meshing.

See Notebook 8 for a short mesh-deformation example.
